<a href="https://colab.research.google.com/github/bmutua/E-Commerce-Shopping-Application/blob/main/Fall_2024_IWE_Voice_Cloner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip3 install -U scipy # because the scipy version packaged with colab is not tolerant of misformated WAV files.

!git clone https://github.com/jnordberg/tortoise-tts.git
%cd tortoise-tts
!pip3 install -r requirements.txt
!pip3 install transformers==4.19.0 einops==0.5.0 rotary_embedding_torch==0.1.5 unidecode==1.3.5
!python3 setup.py install

Cloning into 'tortoise-tts'...
remote: Enumerating objects: 1481, done.
remote: Total 1481 (delta 0), reused 0 (delta 0), pack-reused 1481 (from 1)
Receiving objects: 100% (1481/1481), 53.56 MiB | 22.09 MiB/s, done.
Resolving deltas: 100% (604/604), done.
/content/tortoise-tts/tortoise-tts
  Using cached numba-0.48.0.tar.gz (2.0 MB)
  Preparing metadata (setup.py) ... done
  Using cached ffmpeg-1.4-py3-none-any.whl
  Using cached llvmlite-0.31.0.tar.gz (110 kB)
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of librosa to determine which version is compatible with other requirements. This could take a while.
  Using cached librosa-0.10.2-py3-none-any.whl.metadata (8.6 kB)
  Using cached librosa-0.10.1-py3-none-any.whl.metadata (8.3 kB)
  Using cached librosa-0.10.0.post2-py3-none-any.whl.metadata (8.3 kB)
  Using cached librosa-0.10.0.post1-py3-none-any.whl.metadata (8.3 kB)
  Using cached librosa-0.10.0-py3-none-any.whl.metadata (8.3 kB)
  Using cach

In [ ]:
# Loading necessary libraries
import torch
import torchaudio
import torch.nn as nn
import torch.nn.functional as F

import IPython

from tortoise.api import TextToSpeech
from tortoise.utils.audio import load_audio, load_voice, load_voices

tts = TextToSpeech() # The code provided facilitates the download of all the models utilized by Tortoise from the HuggingFace model hub.

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:392: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(checkpoint_file, map_location="cp

100% (1716988501 of 1716988501) |########| Elapsed Time: 0:01:55 Time:  0:01:55


Done.


100% (60938957 of 60938957) |############| Elapsed Time: 0:00:05 Time:  0:00:05


Done.


...

100% (25193729 of 25193729) |############| Elapsed Time: 0:00:00 Time:  0:00:00


Done.


100% (100715777 of 100715777) |##########| Elapsed Time: 0:00:07 Time:  0:00:07


Done.


/content/tortoise-tts/tortoise/api.py:196: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.autoregressive.load_state_dict(torch.load(f'{models_dir}/autoregressive.pth'))


In [ ]:
text = "Mutua is the best President for Kenyans. He is the best candidate to choose." # For testing I've put that text you can change it for your own use case

preset = "high_quality" # Pick a "preset mode" to determine quality. Options: {"ultra_fast", "fast" (default), "standard", "high_quality"}

In [ ]:
CUSTOM_VOICE_NAME = "Trump" # You upload at least 2 audio clips. They must be a WAV file, 6-10 seconds long.

import os
from google.colab import files

custom_voice_folder = f"tortoise/voices/{CUSTOM_VOICE_NAME}"
# Check if the directory exists before trying to create it.
if not os.path.exists(custom_voice_folder):
  os.makedirs(custom_voice_folder)
for i, file_data in enumerate(files.upload().values()):
  with open(os.path.join(custom_voice_folder, f'{i}.wav'), 'wb') as f:
    f.write(file_data)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Define the channel reducer
class ChannelReducer(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ChannelReducer, self).__init__()
        self.conv = nn.Conv1d(in_channels, out_channels, kernel_size=1)

    def forward(self, x):
        return self.conv(x)

# Initialize the channel reducer
# Adjust the in_channels to match the actual number of channels in voice_samples
channel_reducer = ChannelReducer(in_channels=1, out_channels=80)

# Load voice samples
voice_samples, conditioning_latents = load_voice(CUSTOM_VOICE_NAME)

# Pad shorter tensors to match the length of the longest one
max_length = max([sample.shape[-1] for sample in voice_samples])
padded_samples = []
for sample in voice_samples:
    if sample.shape[-1] < max_length:
        # Pad only the time dimension
        padded_sample = F.pad(sample, (0, max_length - sample.shape[-1]))
    else:
        padded_sample = sample
    padded_samples.append(padded_sample)

# Stack the tensors along a new dimension (creating a batch dimension)
voice_samples = torch.stack(padded_samples, dim=0)

# Apply channel reduction
reduced_voice_samples = channel_reducer(voice_samples)

gen = tts.tts_with_preset(text, voice_samples=voice_samples, conditioning_latents=conditioning_latents,
                          preset=preset)
torchaudio.save(f'generated-{CUSTOM_VOICE_NAME}.wav', gen.squeeze(0).cpu(), 24000)
IPython.display.Audio(f'generated-{CUSTOM_VOICE_NAME}.wav')

In [ ]:
text='Mutua is a legend.'
gen = tts.tts_with_preset(text, voice_samples=voice_samples, conditioning_latents=conditioning_latents,
                          preset=preset)
torchaudio.save(f'generated-{CUSTOM_VOICE_NAME}.wav', gen.squeeze(0).cpu(), 24000)
IPython.display.Audio(f'generated-{CUSTOM_VOICE_NAME}.wav')